In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/lower_grade_glioma_and_glioblastoma/GSE24072'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"EXPRESSION OF VAV1 IN GLIOBLASTOMA MULTIFORME"
!Series_summary	"Background: Even though much progress has been made in the understanding of the molecular nature of glioma, the survival rates of patients affected of this tumour have not changed significantly during these years. Thus, a deeper understanding of this malignancy is still needed in order to predict its outcome and improve patient treatment. Here, we report that VAV1, a GDP/GTP exchange factor for Rho/Rac proteins with oncogenic potential that is involved in the regulation of cytoskeletal dynamics and cell migration."
!Series_summary	"Methodology/Principal Findings: VAV1 is overexpressed in 32 patients diagnosed with high-grade glioma. Such overexpression is linked to the parallel upregulation of a number of genes coding for proteins also involved in cell invasion- and migration-related processes. Unexpectedly, immunohistochemical experiments revealed that VAV1 is not expressed in glioma

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Set is_gene_available based on dataset information
is_gene_available = True

# Determine availability and data type conversion for `lower_grade_glioma_and_glioblastoma` (trait)
trait_row = 2
def convert_trait(value):
    data_map = {
        'glioma grade III': 0,
        'glioma grade IV': 1,
        'glioma grade V': 1  # Assuming higher grades are similar in severity
    }
    trait_value = value.split(': ')[1]
    return data_map.get(trait_value, None)

# Determine availability and data type conversion for `age`
age_row = 1
def convert_age(value):
    try:
        return float(value.split(': ')[1])
    except ValueError:
        return None

# Determine availability and data type conversion for `gender`
gender_row = 0
def convert_gender(value):
    gender_map = {
        'male': 1,
        'female': 0
    }
    gender_value = value.split(': ')[1]
    return gender_map.get(gender_value, None)

save_cohort_info('GSE24072', './preprocessed/lower_grade_glioma_and_glioblastoma/cohort_info.json', is_gene_available, trait_row is not None)

if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'lower_grade_glioma_and_glioblastoma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/lower_grade_glioma_and_glioblastoma/trait_data/GSE24072.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM590744': [0.0, 72.0, 1.0], 'GSM590745': [1.0, 70.0, 0.0], 'GSM590746': [1.0, 34.0, 1.0], 'GSM590747': [1.0, 54.0, 1.0], 'GSM590748': [1.0, 34.0, 1.0], 'GSM591700': [0.0, 68.0, 1.0], 'GSM591701': [0.0, 30.0, 0.0], 'GSM591702': [1.0, 60.0, 1.0], 'GSM591703': [1.0, 73.0, 1.0], 'GSM591704': [1.0, 52.0, 0.0], 'GSM591705': [1.0, 65.0, 1.0], 'GSM591706': [1.0, 70.0, 1.0], 'GSM591707': [1.0, 76.0, 1.0], 'GSM591722': [1.0, 51.0, 1.0], 'GSM591792': [1.0, 73.0, 0.0], 'GSM591793': [1.0, 70.0, 0.0], 'GSM591796': [1.0, 43.0, 1.0], 'GSM591798': [1.0, 67.0, 0.0], 'GSM591800': [1.0, 66.0, 0.0], 'GSM591808': [1.0, 69.0, 1.0], 'GSM591809': [1.0, 74.0, 0.0], 'GSM591811': [1.0, 43.0, 1.0], 'GSM591812': [1.0, 69.0, 0.0], 'GSM591815': [1.0, 36.0, 0.0], 'GSM591817': [0.0, 38.0, 0.0], 'GSM591818': [1.0, 63.0, 1.0], 'GSM591819': [1.0, 74.0, 1.0], 'GSM591829': [1.0, 67.0, 0.0], 'GSM591830': [1.0, 54.0, 1.0], 'GSM591831': [0.0, 46.0, 1.0], 'GSM591832': [0.0, 55.0, 0.0], 'GSM591833': [0.0, 72.0, 1.0]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at',
       '179_at', '1861_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
if requires_gene_mapping:
    # 1. Determine the keys for probe IDs and gene symbols
    identifier_key = 'ID'
    gene_symbol_key = 'Gene Symbol'

    # 2. Get the mapping between probe IDs and genes
    gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

    # 3. Apply the mapping to get the gene expression dataframe
    gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/lower_grade_glioma_and_glioblastoma/gene_data/GSE24072.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data are severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'lower_grade_glioma_and_glioblastoma')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE24072', './preprocessed/lower_grade_glioma_and_glioblastoma/cohort_info.json', True, True, trait_biased, unbiased_merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/lower_grade_glioma_and_glioblastoma/GSE24072.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'lower_grade_glioma_and_glioblastoma', the least common label is '0.0' with 7 occurrences. This represents 21.88% of the dataset.
The distribution of the feature 'lower_grade_glioma_and_glioblastoma' in this dataset is fine.

Quartiles for 'Age':
  25%: 49.75
  50% (Median): 65.5
  75%: 70.0
Min: 30.0
Max: 76.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 13 occurrences. This represents 40.62% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

